# 04 - Clustering

In [3]:
import os, glob, re, warnings, time
from functools import reduce

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.feature_extraction.text import TfidfVectorizer

warnings.filterwarnings("ignore")

# ===== HDBSCAN (required for the HDBSCAN path) =====
try:
    import hdbscan
    HAS_HDBSCAN = True
except Exception:
    HAS_HDBSCAN = False

In [4]:
# =========================
# SPEED / QUALITY TOGGLES
# =========================
DATA_DIR = r"D:/HealthAI Project/data"   # folder that already contains CSVs (already extracted)

DO_PCA = True
PCA_COMPONENTS_MAX = 10
TFIDF_MAX_FEATURES = 50

# Hyperparameter sweep runs on a subset only:
SEARCH_SUBSAMPLE_MAX_N = 15000
SEARCH_GRID = [
    {"min_cluster_size": 40,  "min_samples": None, "metric": "euclidean", "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 50,  "min_samples": None, "metric": "euclidean", "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 50,  "min_samples": 5,    "metric": "euclidean", "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 50,  "min_samples": None, "metric": "euclidean",    "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 40,  "min_samples": 20,   "metric": "euclidean", "cluster_selection_epsilon": 0.1, "cluster_selection_method": "leaf"},
    {"min_cluster_size": 60,  "min_samples": 30,   "metric": "euclidean", "cluster_selection_epsilon": 0.1, "cluster_selection_method": "leaf"},
    {"min_cluster_size": 100, "min_samples": None, "metric": "euclidean", "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 100, "min_samples": 10,   "metric": "euclidean", "cluster_selection_epsilon": 0.0},
    {"min_cluster_size": 50,  "min_samples": 5,    "metric": "euclidean", "cluster_selection_epsilon": 0.5},
    {"min_cluster_size": 100, "min_samples": 10,   "metric": "euclidean", "cluster_selection_epsilon": 0.5},
]

# Final HDBSCAN fit: fit on subset, then label *all* points via approximate_predict
FINAL_SUBSAMPLE_MAX_N = 20000
HDBSCAN_KW = dict(
    cluster_selection_method="eom",  # may be overridden per-cfg
    prediction_data=True,
    approx_min_span_tree=True,
    gen_min_span_tree=False,
    core_dist_n_jobs=-1
)

# Flag "borderline" core points by membership strength
BORDERLINE_THRESH = 0.20

np.set_printoptions(suppress=True)
pd.set_option("display.max_columns", 120)

# =========================
# File helpers
# =========================
def find_one_any(base_dir, rel_candidates, must=False, friendly=""):
    rel_candidates = [c.replace("\\", "/") for c in rel_candidates]
    want = {os.path.basename(c).lower() for c in rel_candidates}
    for root, _, files in os.walk(base_dir):
        for f in files:
            if f.lower() in want:
                path = os.path.join(root, f)
                print(f"[found] {friendly or 'file'} -> {path}")
                return path
    if must:
        raise FileNotFoundError(f"Could not find one of {rel_candidates} under {base_dir}")
    return None

def read_csv_auto(path):
    if path is None: return None
    comp = "gzip" if str(path).lower().endswith(".gz") else None
    return pd.read_csv(path, compression=comp, low_memory=False)

def print_header(title):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))

def coalesce_subject_id(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure a single 'subject_id' column exists, combining _x/_y if present."""
    if "subject_id" in df.columns:
        return df
    s1 = df["subject_id_x"] if "subject_id_x" in df.columns else pd.Series(index=df.index, dtype="float")
    s2 = df["subject_id_y"] if "subject_id_y" in df.columns else pd.Series(index=df.index, dtype="float")
    if ("subject_id_x" in df.columns) or ("subject_id_y" in df.columns):
        df["subject_id"] = s1.combine_first(s2)
        drop_cols = [c for c in ["subject_id_x","subject_id_y"] if c in df.columns]
        df = df.drop(columns=drop_cols)
    return df

def _metric_supported(metric: str) -> bool:
    return metric in {"euclidean", "l1", "l2", "manhattan", "cityblock", "cosine"}

In [5]:
# =========================
# Pick up CSVs
# =========================
patients_p   = find_one_any(DATA_DIR, ["patients.csv"])
admissions_p = find_one_any(DATA_DIR, ["admissions.csv"])
vitals_p     = find_one_any(DATA_DIR, ["vitalsign.csv"])
labevents_p  = find_one_any(DATA_DIR, ["labevents.csv"])
d_labitems_p = find_one_any(DATA_DIR, ["d_labitems.csv"])
presc_p      = find_one_any(DATA_DIR, ["prescriptions.csv"])
pharmacy_p   = find_one_any(DATA_DIR, ["pharmacy.csv"])

patients   = read_csv_auto(patients_p)
admissions = read_csv_auto(admissions_p)
vitals     = read_csv_auto(vitals_p)
labevents  = read_csv_auto(labevents_p)
d_labitems = read_csv_auto(d_labitems_p)
presc      = read_csv_auto(presc_p)
pharmacy   = read_csv_auto(pharmacy_p)

print_header("Files present")
print({
    "patients": patients is not None,
    "admissions": admissions is not None,
    "vitals": vitals is not None,
    "labevents": labevents is not None,
    "d_labitems": d_labitems is not None,
    "prescriptions": presc is not None,
    "pharmacy": pharmacy is not None,
})

[found] file -> D:/HealthAI Project/data\MIMIC IV\patients.csv
[found] file -> D:/HealthAI Project/data\MIMIC IV\admissions.csv
[found] file -> D:/HealthAI Project/data\MIMIC IV\vitalsign.csv
[found] file -> D:/HealthAI Project/data\MIMIC IV\labevents.csv
[found] file -> D:/HealthAI Project/data\MIMIC IV\d_labitems.csv
[found] file -> D:/HealthAI Project/data\MIMIC IV\prescriptions.csv
[found] file -> D:/HealthAI Project/data\MIMIC IV\pharmacy.csv

Files present
{'patients': True, 'admissions': True, 'vitals': True, 'labevents': True, 'd_labitems': True, 'prescriptions': True, 'pharmacy': True}


In [6]:
# =========================
# Data preparation
# =========================
t0 = time.time()
adm = admissions.copy()
for c in ("admittime", "dischtime"):
    if c in adm.columns:
        adm[c] = pd.to_datetime(adm[c], errors="coerce")
adm["los_days"] = (adm["dischtime"] - adm["admittime"]).dt.total_seconds()/86400.0

pat = patients.copy()
if "sex" not in pat.columns and "gender" in pat.columns:
    pat = pat.rename(columns={"gender": "sex"})
for c in ("anchor_year", "anchor_age"):
    if c in pat.columns:
        pat[c] = pd.to_numeric(pat[c], errors="coerce")

# Map patient attrs to admissions rows
anchor_year_s = pat.set_index("subject_id")["anchor_year"] if "anchor_year" in pat.columns else None
anchor_age_s  = pat.set_index("subject_id")["anchor_age"]  if "anchor_age"  in pat.columns else None
dob_s         = pat.set_index("subject_id")["dob"]          if "dob"         in pat.columns else None

ay = adm["subject_id"].map(anchor_year_s) if anchor_year_s is not None else None
aa = adm["subject_id"].map(anchor_age_s)  if anchor_age_s  is not None else None
db = adm["subject_id"].map(dob_s)         if dob_s         is not None else None

def compute_age_at_admit_safe(admittime, anchor_year=None, anchor_age=None, dob=None):
    y = pd.to_datetime(admittime, errors="coerce").dt.year
    age = pd.Series(np.nan, index=y.index, dtype=float)
    if anchor_year is not None and anchor_age is not None:
        ay = pd.to_numeric(anchor_year, errors="coerce")
        aa = pd.to_numeric(anchor_age,  errors="coerce")
        age = (aa + (y - ay)).astype(float)
    if dob is not None:
        dob_dt = pd.to_datetime(dob, errors="coerce")
        age_from_dob = (pd.to_datetime(admittime, errors="coerce") - dob_dt).dt.total_seconds() / (365.25*24*3600)
        age = age.where(age.notna(), age_from_dob)
    return age.clip(lower=0, upper=120)

adm["age_at_admit"] = compute_age_at_admit_safe(adm["admittime"], ay, aa, db)

# per-patient rollup
adm_subj = adm.groupby("subject_id").agg(
    admits=("hadm_id","nunique"),
    age_at_first_admit=("age_at_admit","min"),
    mean_los_days=("los_days","mean")
).reset_index()

if "sex" in pat.columns:
    adm_subj = adm_subj.merge(pat[["subject_id","sex"]], on="subject_id", how="left")

In [7]:
# =========================
# Vitals (per patient) — ensure numeric before mean()
# =========================
vitals_subj = None
if vitals is not None and "subject_id" in vitals.columns:
    v = vitals.copy()
    rename_map = {
        "heartrate":"heartrate", "heart_rate":"heartrate", "hr":"heartrate",
        "resp_rate":"resprate", "respiratory_rate":"resprate", "rr":"resprate",
        "o2sat":"o2sat", "spo2":"o2sat", "oxygen_saturation":"o2sat",
        "sbp":"sbp", "systolic":"sbp", "systolic_bp":"sbp",
        "dbp":"dbp", "diastolic":"dbp", "diastolic_bp":"dbp",
        "temperature":"temperature", "temp":"temperature",
        "weight":"weight_kg", "weight_kg":"weight_kg",
        "height":"height_cm", "height_cm":"height_cm"
    }
    lowcols = {c: c.lower() for c in v.columns}
    ren = {orig: rename_map[low] for orig, low in lowcols.items() if low in rename_map}
    v = v.rename(columns=ren)
    keep = [c for c in ["temperature","heartrate","resprate","o2sat","sbp","dbp","weight_kg","height_cm"] if c in v.columns]
    if keep:
        for c in keep:
            v[c] = pd.to_numeric(v[c], errors="coerce")
        g = v.groupby("subject_id")[keep].mean()  # numeric-only
        g = g.reset_index()
        if {"weight_kg","height_cm"}.issubset(g.columns):
            h_m = g["height_cm"] / 100.0
            with np.errstate(divide="ignore", invalid="ignore"):
                bmi = g["weight_kg"] / (h_m**2)
            g["bmi"] = bmi.replace([np.inf, -np.inf], np.nan)
        g.columns = ["subject_id"] + [f"vital_{c}_mean" for c in g.columns[1:]]
        vitals_subj = g

In [8]:
# =========================
# Labs (per patient) — numeric guard
# =========================
lab_subj = None
if (labevents is not None) and (d_labitems is not None):
    dli = d_labitems.copy()
    dli["label_l"] = dli["label"].astype(str).str.lower()
    lab_keep = dli[dli["label_l"].str.contains(
        r"glucose|hba1c|hemoglobin|creatinine|cholesterol|triglyceride|hdl|ldl",
        regex=True, na=False
    )][["itemid","label_l"]]

    le = labevents.copy().dropna(subset=["itemid","valuenum"])
    le["valuenum"] = pd.to_numeric(le["valuenum"], errors="coerce")
    le = le[le["itemid"].isin(lab_keep["itemid"])]
    le = le.merge(adm[["hadm_id","subject_id"]], on="hadm_id", how="left")
    le = coalesce_subject_id(le).dropna(subset=["subject_id"])
    le["subject_id"] = pd.to_numeric(le["subject_id"], errors="coerce")

    le = le.merge(lab_keep, on="itemid", how="left")
    lab_subj = (
        le.groupby(["subject_id","label_l"])["valuenum"].mean()
          .unstack(fill_value=np.nan)
          .reset_index()
    )
    lab_subj.columns = ["subject_id"] + [f"lab_{str(c).replace(' ','_')}_mean" for c in lab_subj.columns[1:]]

In [9]:
# =========================
# Med history (per patient)
# =========================
med_subj = None
if presc is not None and "drug" in presc.columns:
    rx = presc.copy()
    rx["drug"] = rx["drug"].astype(str).str.lower()
    rx = rx.merge(adm[["hadm_id","subject_id"]], on="hadm_id", how="left")
    rx = coalesce_subject_id(rx).dropna(subset=["subject_id"])
    rx["subject_id"] = pd.to_numeric(rx["subject_id"], errors="coerce")

    rx_count = rx.groupby("subject_id").agg(
        rx_count=("drug","count"),
        rx_unique=("drug","nunique")
    ).reset_index()

    def flag_any(series: pd.Series, keywords):
        if series.empty: return 0
        pat = r"(" + r"|".join(re.escape(k) for k in keywords) + r")"
        return int(series.str.contains(pat, na=False).any())

    rows = []
    for sid, g in rx.groupby("subject_id"):
        s = g["drug"]
        rows.append({
            "subject_id": sid,
            "rx_insulin":   flag_any(s, ["insulin"]),
            "rx_metformin": flag_any(s, ["metformin"]),
            "rx_statin":    flag_any(s, ["atorvastatin","rosuvastatin","simvastatin","pravastatin","lovastatin"])
        })
    rx_flags = pd.DataFrame(rows)
    med_subj = rx_count.merge(rx_flags, on="subject_id", how="outer").fillna(0)

In [10]:
# =========================
# “Notes” proxy via medication text (per patient)
# =========================
tok_subj = None
if pharmacy is not None and {"hadm_id","medication"}.issubset(pharmacy.columns):
    ph = pharmacy[["hadm_id","medication"]].copy()
    ph["medication"] = ph["medication"].astype(str)
    ph = ph.merge(adm[["hadm_id","subject_id"]], on="hadm_id", how="left")
    ph = coalesce_subject_id(ph).dropna(subset=["subject_id"])
    ph["subject_id"] = pd.to_numeric(ph["subject_id"], errors="coerce")

    text_per_subject = ph.groupby("subject_id")["medication"].apply(lambda s: " ".join(s)).reset_index()
    text_per_subject["medication"] = text_per_subject["medication"].fillna("")
    if len(text_per_subject) > 0:
        tfidf = TfidfVectorizer(
            max_features=TFIDF_MAX_FEATURES,
            stop_words="english",
            token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z\-]+\b"
        )
        Xtf = tfidf.fit_transform(text_per_subject["medication"])
        tok_subj = pd.DataFrame(Xtf.toarray(), columns=[f"tok_{t}" for t in tfidf.get_feature_names_out()])
        tok_subj.insert(0, "subject_id", text_per_subject["subject_id"].values)

print_header(f"Feature engineering finished in {time.time()-t0:.1f}s")


Feature engineering finished in 172.5s


In [11]:
# =========================
# Assemble feature table (per patient)
# =========================
frames = [adm_subj]
for f in [vitals_subj, lab_subj, med_subj, tok_subj]:
    if f is not None:
        frames.append(f)

feat = reduce(lambda L, R: pd.merge(L, R, on="subject_id", how="outer"), frames).copy()

# Handle sex (if present) as one-hot
if "sex" in feat.columns:
    feat["sex"] = feat["sex"].fillna("Unknown")
    sex_ohe = pd.get_dummies(feat["sex"], prefix="sex", dummy_na=False)
    feat = pd.concat([feat.drop(columns=["sex"]), sex_ohe], axis=1)

# Build numeric matrix; drop ID
num_df = feat.select_dtypes(include=np.number).copy()
if "subject_id" in num_df.columns:
    num_df = num_df.drop(columns=["subject_id"])

# Fill numeric NaNs with medians and tame outliers
num_df = num_df.apply(lambda s: s.fillna(s.median()), axis=0)
q_low  = num_df.quantile(0.005)
q_high = num_df.quantile(0.995)
num_df = num_df.clip(q_low, q_high, axis=1)

# Scale
scaler = StandardScaler()
X = scaler.fit_transform(num_df.values)

# Optional PCA
n_comp = min(PCA_COMPONENTS_MAX, X.shape[1])
if DO_PCA and n_comp >= 2:
    pca = PCA(n_components=n_comp, random_state=42)
    X_use = pca.fit_transform(X)
    print_header(f"PCA: n_components={n_comp}, explained variance={pca.explained_variance_ratio_.sum():.3f}")
else:
    pca = None
    X_use = X

N = X_use.shape[0]
rng = np.random.default_rng(42)


PCA: n_components=10, explained variance=0.429


In [ ]:
# =========================
# HDBSCAN
# =========================
if not HAS_HDBSCAN:
    print_header("HDBSCAN not installed — skipping HDBSCAN and using K-Means only.")
    labels_all = np.full(N, -1, dtype=int)
    strengths = np.zeros(N, dtype=float)
    best_cfg = None
    sil_full = ch_full = np.nan
    noise_frac_full = 1.0
else:
    print_header("HDBSCAN")
    t1 = time.time()

    # 0) Filter/repair grid for unsupported metrics
    fixed_grid = []
    for cfg in SEARCH_GRID:
        cfg = cfg.copy()
        m = cfg.get("metric", "euclidean")
        if not _metric_supported(m):
            print(f"[info] Metric '{m}' not supported; forcing 'euclidean' for cfg={cfg}.")
            cfg["metric"] = "euclidean"
        fixed_grid.append(cfg)

    # 1) Build subset for search
    if N > SEARCH_SUBSAMPLE_MAX_N:
        idx_search = rng.choice(N, SEARCH_SUBSAMPLE_MAX_N, replace=False)
    else:
        idx_search = np.arange(N)

    best_score = None  # (sil, ch, -noise_frac)
    best_cfg   = None

    for cfg in fixed_grid:
        metric = cfg.get("metric", "euclidean")
        method = cfg.get("cluster_selection_method", HDBSCAN_KW["cluster_selection_method"])

        # prepare space for this metric
        X_search = X_use[idx_search]
        if metric == "cosine":
            X_space = normalize(X_search)  # L2-normalize rows
            sil_metric = "cosine"
        else:
            X_space = X_search
            sil_metric = "euclidean"

        cl = hdbscan.HDBSCAN(
            min_cluster_size=cfg["min_cluster_size"],
            min_samples=cfg["min_samples"],
            metric=metric,
            cluster_selection_epsilon=cfg.get("cluster_selection_epsilon", 0.0),
            cluster_selection_method=method,
            **{k:v for k,v in HDBSCAN_KW.items() if k != "cluster_selection_method"}
        )
        labels_sub = cl.fit_predict(X_space)
        core_mask = labels_sub >= 0
        if core_mask.sum() < 2 or len(np.unique(labels_sub[core_mask])) < 2:
            continue

        sil = silhouette_score(X_space[core_mask], labels_sub[core_mask], metric=sil_metric)
        ch  = calinski_harabasz_score(X_space[core_mask], labels_sub[core_mask])
        noise_frac = float((labels_sub < 0).sum()) / len(labels_sub)
        score = (sil, ch, -noise_frac)

        print(f"cfg={cfg} -> Sil={sil:.3f}, CH={ch:.1f}, noise={noise_frac:.3f}")

        def _safe_tuple(t):  # handle NaNs
            return tuple(x if np.isfinite(x) else -np.inf for x in t)
        if (best_score is None) or (_safe_tuple(score) > _safe_tuple(best_score)):
            best_score = score
            best_cfg   = cfg

    if best_cfg is None:
        best_cfg = {"min_cluster_size": 50, "min_samples": None, "metric": "euclidean", "cluster_selection_epsilon": 0.0}
        print(f"[warn] No valid clustering found in sweep; falling back to {best_cfg}")

    # 2) Final model fit on a (slightly bigger) subset — respect metric space
    if N > FINAL_SUBSAMPLE_MAX_N:
        idx_final = rng.choice(N, FINAL_SUBSAMPLE_MAX_N, replace=False)
    else:
        idx_final = np.arange(N)

    metric_final = best_cfg.get("metric", "euclidean")
    method_final = best_cfg.get("cluster_selection_method", HDBSCAN_KW["cluster_selection_method"])

    X_finalfit = X_use[idx_final]
    X_all_eval = X_use.copy()
    if metric_final == "cosine":
        X_finalfit = normalize(X_finalfit)
        X_all_eval = normalize(X_all_eval)
        sil_metric_final = "cosine"
    else:
        sil_metric_final = "euclidean"

    final_cl = hdbscan.HDBSCAN(
        min_cluster_size=best_cfg["min_cluster_size"],
        min_samples=best_cfg["min_samples"],
        metric=metric_final,
        cluster_selection_epsilon=best_cfg.get("cluster_selection_epsilon", 0.0),
        cluster_selection_method=method_final,
        **{k:v for k,v in HDBSCAN_KW.items() if k != "cluster_selection_method"}
    )
    final_cl.fit(X_finalfit)

    # 3) Label ALL points via approximate_predict (in same metric space)
    labels_all, strengths = hdbscan.approximate_predict(final_cl, X_all_eval)

    # Metrics on core-only (exclude noise) in same space
    mask_core = labels_all >= 0
    n_clusters_core = len(np.unique(labels_all[mask_core])) if mask_core.sum() > 1 else 0
    if mask_core.sum() > 1 and n_clusters_core >= 2:
        sil_full = silhouette_score(X_all_eval[mask_core], labels_all[mask_core], metric=sil_metric_final)
        ch_full  = calinski_harabasz_score(X_all_eval[mask_core], labels_all[mask_core])
    else:
        sil_full, ch_full = np.nan, np.nan

    noise_frac_full = float((labels_all < 0).sum()) / len(labels_all)

    print(f"\nBest config: {best_cfg}")
    print(f"Clusters: {n_clusters_core}")
    print(f"Noise fraction: {noise_frac_full:.3f}")
    print(f"Silhouette: {np.nan if not np.isfinite(sil_full) else round(sil_full, 3)}")
    print(f"Calinski–Harabasz: {np.nan if not np.isfinite(ch_full) else round(ch_full, 1)}")
    print(f"HDBSCAN total time: {time.time()-t1:.1f}s")


HDBSCAN
cfg={'min_cluster_size': 40, 'min_samples': None, 'metric': 'euclidean', 'cluster_selection_epsilon': 0.0} -> Sil=0.674, CH=5572.0, noise=0.529
cfg={'min_cluster_size': 50, 'min_samples': None, 'metric': 'euclidean', 'cluster_selection_epsilon': 0.0} -> Sil=0.755, CH=6805.9, noise=0.524
cfg={'min_cluster_size': 50, 'min_samples': 5, 'metric': 'euclidean', 'cluster_selection_epsilon': 0.0} -> Sil=0.582, CH=3055.8, noise=0.576
cfg={'min_cluster_size': 50, 'min_samples': None, 'metric': 'euclidean', 'cluster_selection_epsilon': 0.0} -> Sil=0.755, CH=6805.9, noise=0.524
cfg={'min_cluster_size': 40, 'min_samples': 20, 'metric': 'euclidean', 'cluster_selection_epsilon': 0.1, 'cluster_selection_method': 'leaf'} -> Sil=0.436, CH=2940.6, noise=0.733
cfg={'min_cluster_size': 60, 'min_samples': 30, 'metric': 'euclidean', 'cluster_selection_epsilon': 0.1, 'cluster_selection_method': 'leaf'} -> Sil=0.437, CH=5339.0, noise=0.750
cfg={'min_cluster_size': 100, 'min_samples': None, 'metric': '

In [30]:
# =========================
# Results & Profiles
# =========================
# Attach labels to subjects
assign = feat[["subject_id"]].copy()
assign["hdb_cluster"]  = labels_all                  # hdbscan labels (noise = -1)
assign["hdb_strength"] = strengths
assign["hdb_borderline"] = ((assign["hdb_cluster"] >= 0) & (assign["hdb_strength"] < BORDERLINE_THRESH)).astype(int)

# Profile table
profile_df = feat.merge(assign[["subject_id","hdb_cluster","hdb_strength","hdb_borderline"]],
                        on="subject_id", how="left")

def pick_cols(df):
    want_prefixes = [
        "age_at_first_admit", "admits", "mean_los_days",
        "vital_sbp_mean", "vital_dbp_mean", "vital_heartrate_mean", "vital_o2sat_mean",
        "vital_bmi_mean",
        "lab_glucose", "lab_hba1c", "lab_cholesterol", "lab_hdl", "lab_ldl", "lab_triglyceride",
        "lab_creatinine", "lab_hemoglobin",
        "rx_count", "rx_unique", "rx_insulin", "rx_metformin", "rx_statin",
        "sex_"
    ]
    cols = []
    for pref in want_prefixes:
        found = [c for c in df.columns if c.startswith(pref)]
        if pref == "sex_":
            cols += found
        elif found:
            cols.append(found[0])
    cols = ["hdb_cluster", "hdb_strength", "hdb_borderline"] + list(dict.fromkeys(cols))
    return cols

cols = [c for c in pick_cols(profile_df) if c in profile_df.columns]

# ---- HDBSCAN profiles (exclude noise, ensure numeric for medians) ----
hdb_base = profile_df[profile_df["hdb_cluster"] >= 0].copy()

# Coerce key numeric columns used in agg (if present)
for col in ["age_at_first_admit", "admits", "mean_los_days"]:
    if col in hdb_base.columns:
        hdb_base[col] = pd.to_numeric(hdb_base[col], errors="coerce")

hdb_prof = (hdb_base
            .groupby("hdb_cluster")
            .agg(
                n=("subject_id","count"),
                borderline_pct=("hdb_borderline", lambda s: 100.0*pd.to_numeric(s, errors="coerce").mean()),
                mean_strength=("hdb_strength", lambda s: pd.to_numeric(s, errors="coerce").mean()),
                age_med=("age_at_first_admit","median") if "age_at_first_admit" in hdb_base.columns else ("subject_id","size"),
                admits_med=("admits","median") if "admits" in hdb_base.columns else ("subject_id","size"),
                los_med=("mean_los_days","median") if "mean_los_days" in hdb_base.columns else ("subject_id","size"),
            ))

# Add lab medians safely
for lab_col in [c for c in hdb_base.columns if c.startswith("lab_")]:
    hdb_base[lab_col] = pd.to_numeric(hdb_base[lab_col], errors="coerce")
    hdb_prof[f"{lab_col}_med"] = hdb_base.groupby("hdb_cluster")[lab_col].median()

print_header("HDBSCAN cluster profiles (medians/percentages)")
try:
    from IPython.display import display
    display(hdb_prof)
except Exception:
    print(hdb_prof.to_string())


HDBSCAN cluster profiles (medians/percentages)


,n,borderline_pct,mean_strength,age_med,admits_med,los_med,lab_%_hemoglobin_a1c_mean_med,lab_24_hr_creatinine_mean_med,"lab_albumin/creatinine,_urine_mean_med","lab_amylase/creatinine_ratio,_urine_mean_med",lab_carboxyhemoglobin_mean_med,lab_cholesterol_ratio_(total/hdl)_mean_med,"lab_cholesterol,_ascites_mean_med","lab_cholesterol,_body_fluid_mean_med","lab_cholesterol,_hdl_mean_med","lab_cholesterol,_ldl,_calculated_mean_med","lab_cholesterol,_ldl,_measured_mean_med","lab_cholesterol,_pleural_mean_med","lab_cholesterol,_total_mean_med",lab_creatinine_mean_med,lab_creatinine_clearance_mean_med,"lab_creatinine,_ascites_mean_med","lab_creatinine,_body_fluid_mean_med","lab_creatinine,_joint_fluid_mean_med","lab_creatinine,_pleural_mean_med","lab_creatinine,_serum_mean_med","lab_creatinine,_urine_mean_med","lab_creatinine,_whole_blood_mean_med",lab_fetal_hemoglobin_mean_med,lab_glucose_mean_med,"lab_glucose,_ascites_mean_med","lab_glucose,_body_fluid_mean_med","lab_glucose,_csf_mean_med","lab_glucose,_joint_fluid_mean_med","lab_glucose,_pleural_mean_med","lab_glucose,_urine_mean_med",lab_hemoglobin_mean_med,lab_hemoglobin_a2_mean_med,lab_hemoglobin_c_mean_med,lab_hemoglobin_f_mean_med,lab_hemoglobin_other_mean_med,"lab_hemoglobin,_calculated_mean_med",lab_methemoglobin_mean_med,lab_p50_of_hemoglobin_mean_med,lab_protein/creatinine_ratio_mean_med,lab_triglycerides_mean_med,"lab_triglycerides,_ascites_mean_med","lab_triglycerides,_pleural_mean_med",lab_urine_creatinine_mean_med
hdb_cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,11158,0.000000,0.777461,65.0,1.0,5.660417,5.600000,1106.5,20.966667,6.7,2.0,3.25,45.5,88.0,51.833333,93.0,102.00,47.75,168.583333,0.819512,59.0,0.8,1.3,NaN,0.7,1.30,76.0,0.9,0.0,116.3125,121.333333,82.0,70.0,113.0,110.0,17.0,10.950000,2.5,0.0,0.0,0.90,13.75,0.0,NaN,0.3,112.50,71.5,30.25,52.0
1,4965,0.000000,0.758998,32.0,1.0,2.660764,5.333333,1300.0,6.900000,NaN,1.5,3.10,NaN,NaN,59.000000,102.0,110.75,NaN,181.000000,0.650000,113.0,NaN,1.6,NaN,NaN,0.75,79.0,0.7,0.0,95.0000,NaN,34.5,60.0,NaN,NaN,NaN,12.180000,2.5,0.0,0.0,0.95,14.10,0.0,NaN,0.2,88.00,NaN,NaN,69.0
2,3374,0.000000,0.967372,42.0,1.0,2.116319,5.500000,1422.0,9.100000,NaN,2.0,3.20,NaN,NaN,58.000000,103.0,110.00,90.00,183.000000,0.800000,159.0,1.0,5.9,NaN,0.7,0.60,90.0,0.9,NaN,102.5000,59.000000,NaN,61.0,17.5,104.0,NaN,12.766667,2.4,0.0,0.0,NaN,14.25,0.0,NaN,0.1,100.55,33.0,NaN,73.5
3,141872,8.005808,0.679673,41.0,1.0,0.543056,5.600000,1205.0,9.900000,3.4,2.0,3.30,NaN,NaN,56.000000,106.0,117.50,70.00,189.041667,0.825000,96.0,0.7,NaN,NaN,0.7,1.00,106.0,0.9,0.0,98.0000,120.000000,57.0,63.0,80.0,94.0,5.0,13.600000,2.5,0.0,0.0,57.85,14.30,0.0,NaN,0.1,105.50,NaN,30.00,80.0


In [ ]:
# =========================
# Save outputs
# =========================
OUT_DIR = "Models/Cluster_Outputs"
os.makedirs(OUT_DIR, exist_ok=True)
feat.to_csv(os.path.join(OUT_DIR, "patient_features.csv"), index=False)
assign.to_csv(os.path.join(OUT_DIR, "cluster_assignments.csv"), index=False)
hdb_prof.to_csv(os.path.join(OUT_DIR, "clustered_data.csv"))

print_header("Saved")
print({
    "patient_features.csv": os.path.join(OUT_DIR, "patient_features.csv"),
    "cluster_assignments.csv": os.path.join(OUT_DIR, "cluster_assignments.csv"),
    "clustered_data.csv": os.path.join(OUT_DIR, "clustered_data.csv")
})


=====
Saved
=====
{'patient_features.csv': 'cluster_outputs\\patient_features.csv', 'cluster_assignments.csv': 'cluster_outputs\\cluster_assignments.csv', 'hdbscan_profiles.csv': 'cluster_outputs\\hdbscan_profiles.csv'}
